# Fast Fourier Transform (FFT)
The following gives background information that is helpful to understand how the Fast Fourier Transform (FFT) and the corresponding normalizations are defined in pyfar and how these are related to the concepts of energy and power signals.


## FFT definition
The discrete Fourier transform (DFT) of an arbitrary, but band-limited signal $x(n)$ is defined as
$$ X(k) = \sum_{n=0}^{N-1} x(n) e^{-i 2 \pi k \frac{n}{N}}$$
using a negative sign convention in the transform kernel $e^{-i 2 \pi k \frac{n}{N}}$, and normalized angular frequency $\omega_k = 2 \pi k / N$. Analogously, the inverse discrete Fourier transform (IDFT) is implemented as

$$ x(n) = \frac{1}{N} \sum_{k=0}^{N-1} X(k) e^{i2\pi k \frac{n}{N}}$$

The *Fast Fourier Transform* denotes the efficient implementation of DFT and IDFT.

Note that real-valued time signals result in Fourier spectra with complex conjugate symmetry for negative and positive frequencies $X(k) = X(-k)^*$. In this case, the left-hand side of the spectrum can be discarded, and only the right-hand side needs to be saved.

## FFT normalizations
Pyfar implements five normalizations  after [Ahrens at al. (2020)](http://www.aes.org/e-lib/browse.cfm?elib=20838) that can be applied to spectra after the DFT. The normalizations are implicitly used by the [pyfar.Signal class](https://pyfar.readthedocs.io/en/stable/classes/pyfar.audio.html#pyfar.classes.audio.Signal), and are available from [pyfar.dsp.fft.normalization](https://pyfar.readthedocs.io/en/stable/modules/pyfar.dsp.fft.html#pyfar.dsp.fft.normalization). For a Signal object `signal`, `signal.freq` contains the normalized spectrum according to `signal.fft_norm` and `signal.freq_raw` contains the raw spectrum without any normalization. **The time data (**`signal.time`**) does not change regardless of the normalization.**

### Definitions

| Norm        | Equation                                                                                                 |
|-------------|----------------------------------------------------------------------------------------------------------|
| `'none'`    | --                                                                                                       |
| `'unitary'` | $X_{\text{SS}}(k) = \left\{ \begin{array}{ll} X(k) & \forall k=0, k=\frac{N}{2} \\ 2 X(k) & \forall 0<k< \frac{N}{2} \end{array} \right.$|
| `'amplitude'` | $\overline{X}_{\text{SS}}(k) = \frac{1}{N} X_{\text{SS}}(k)$                                          |
| `'rms'`       | $\overline{X}_{RMS}(k) = \left\{ \begin{array}{ll} \frac{1}{\sqrt{2}} \overline{X}_{\text{SS}}(k) & \forall 0<k< \frac{N}{2} \\ \quad \overline{X}_{\text{SS}}(k) & \forall k=0, k=\frac{N}{2} \end{array} \right.$|
| `'power'`     | $\overline{\overline{X}}_{\text{SS}}(k) = \lvert \overline{X}_{\text{RMS}}(k) \lvert ^2$           |
| `'psd'`       | $\overline{\overline{\underline{X}}}_{\text{SS}}(k) = \frac{N}{f_s} \overline{\overline{X}}_{\text{SS}}(k) = \frac{N}{f_s} \lvert \overline{X}_{\text{RMS}}(k) \lvert ^2$|

Note that the above formulation holds for real-valued signals with single-sided spectra $X_{\text{SS}}(k)$. Hence, there are small differences in the definitions compared to the formulas written in [Ahrens et al. (2020)](http://www.aes.org/e-lib/browse.cfm?elib=20838).

### Explanations

* `'none'`:
        Use the spectrum as it is. This norm is to be used for energy signals such as impulse responses (IRs). In this case the spectrum is independent of the signal length. For power signals, the spectrum depends on the number of samples ("longer signal = more energy"), so different normalizations are appropriate.
* `'unitary'`:
        Multiply the spectrum with a factor of 2 in order to represent power related measures correctly (e.g., the amplitude or RMS-value, see below). **All following normalizations make use of this (implicitly assuming real-valued time signals, i.e., signals with single-sided spectrum).**
* `'amplitude'`:
        Normalize the spectrum to show the amplitude of the pure tone components contained in a signal. If the signal is a sine with an amplitude of 1, the spectrum will have an absolute value of 1 (0 dB) at the frequency of the sine.
* `'rms'`:
        Normalize the spectrum to show the RMS value of the pure tone components contained in a signal. If the signal is a sine with an amplitude of 1, the spectrum will have an absolute value of $1/\sqrt{2}$ (-3 dB) at the frequency of the sine.
* `'power'`:
        In a dB representation, it equals the `'rms'` normalization describing a power quantity. For stochastic broadband signals, this results in a dependence of the magnitude on the sampling rate as these are defined by a constant power density (see `'psd'`).
* `'psd'`:
        Using this normalization, signals are represented as *power densities* (e.g. in Pa²/Hz), leading to a meaningful representation for broadband stochastic signals independently of the sampling rate.

### Overview: Appropriate FFT normalizations

| Signal type            | Variation                              | Normalization          |
|------------------------|----------------------------------------|------------------------|
| Energy                 | Impulse responses / transfer functions | `'none'`               |
| Power                  | Discrete tones                         | `'amplitude'`, `'rms'`, `'power'` |
| Power                  | Broadband stochastic signals           | `'psd'`              |

## Examples
Four signals with a length of 100 samples and a sampling rate of 10 kHz are used for illustration.

In [ ]:
import pyfar as pf
import numpy as np
import matplotlib.pyplot as plt


# set number of samples and sampling rate
n_samples = 1e3
sampling_rate = 10e3

An **impulse** ($x(0)=1$ and zero otherwise) with a constant spectrum. This is an energy signal, so the appropriate normalization is `'none'` (the default).

In [ ]:
impulse = pf.signals.impulse(n_samples, sampling_rate=sampling_rate)

A **fractional octave FIR filter** presenting a system with finite energy (e.g., a loudspeaker transfer function, a room impulse response, an HRTF ...). It is an energy signal, so the appropriate normalization is `'none'`.

In [ ]:
fir = pf.dsp.filter.fractional_octave_bands(
    impulse, num_fractions=1, freq_range=(500, 700))

A **sine** signal with an amplitude of $1\,\text{Pa}$. It represents a discrete tone of which a snippet was recorded. Accordingly, it possess a finite power but infinite energy and is a power signal with appropriate normalizations `'amplitude'`, `'rms'`, or `'power'`.

In [ ]:
sine = pf.signals.sine(1e3, n_samples, sampling_rate=sampling_rate)

A **white noise** signal with an RMS value of $1/\sqrt{2}\,\text{Pa}$. It represents a broadband stochastic signal of which a snippet was recorded of. Accordingly, it is a power signal with the appropriate normalization `'psd'`.

In [ ]:
noise = pf.signals.noise(n_samples, rms=1/np.sqrt(2), sampling_rate=sampling_rate)

Let's have a look! Note that the implied units differ and a prefix of 10 is used for the dB calculations of the normalizations `'power'` and `'psd'`.

In [ ]:
# apply pyfar plotstyle
pf.plot.use()
# figure with 3 rows and 2 columns, one plot for each fft_norm
fig, axes = plt.subplots(3, 2, figsize=(10, 10))
fft_norms = ['none', 'unitary', 'amplitude', 'rms', 'power', 'psd']
ylim = [(-20, 70), (-20, 70), (-80, 10), (-80, 10), (-80, 10), (-80, 10)]
# implied units differ between signals and fft_norms
units = [
    '$\\mathrm{1/Hz}$', '$\\mathrm{1/Hz}$', '$\\mathrm{1}$', '$\\mathrm{1}$',
    '$\\mathrm{1}$', '$\mathrm{1}/\\sqrt{\\mathrm{Hz}}$']
# set the log_prefix for the different fft_norms
log_prefixs = [20, 20, 20, 20, 10, 10]
for idx, (fft_norm, unit, log_prefix, ylim, ax) in \
        enumerate(zip(fft_norms, units, log_prefixs, ylim, axes.flatten())):
    # set fft_norm
    fir.fft_norm = fft_norm
    impulse.fft_norm = fft_norm
    sine.fft_norm = fft_norm
    noise.fft_norm = fft_norm
    pf.plot.freq(
        impulse, log_prefix=log_prefix, ax=ax,
        label=f"Impulse in dB re. {unit}")
    pf.plot.freq(
        fir, log_prefix=log_prefix, ax=ax,
        label=f"FIR Filter in dB re. {unit}")
    pf.plot.freq(
        noise, log_prefix=log_prefix, ax=ax,
        label=f"Noise in dB re. {unit.replace('1', '1 Pa')}")
    pf.plot.freq(
        sine, log_prefix=log_prefix, ax=ax,
        label=f"Sine in dB re. {unit.replace('1', '1 Pa')}")
    ax.set_title(f"'{fft_norm}'")
    ax.set_ylim(ylim)
    ax.legend(loc='upper left')

### Explanations

* The `'none'` normalization gives the expected results for the impulse and FIR filter, but leads to a magnitude of number of samples/2 for the sine signal (1000/2, 60-6 dB). As illustrated, other normalizations than `'none'` are not meaningful for these energy signals.
* For the sine signal, `'unitary'` normalization considers the factor 2 due to the single-side spectrum (+6 dB compared to `'none'`). The `'amplitude'` normalization considers the number of samples, so the amplitude of 1 Pa (= 0 dB) is represented in the spectrum. Accordingly, the magnitude of the sine is -3 dB with `'rms'` and `'power'` normalization.
* With `'psd'` normalization, the sine's magnitude is reduced by a factor of number of samples / sampling rate (1/10, -10 dB) compared to  `'rms'`. As discussed above, this normalization is only meaningful for the noise, as is represents a spectral density.

For further details, especially on the background of the power normalizations, it is referred to [Ahrens at al. (2020)](http://www.aes.org/e-lib/browse.cfm?elib=20838). See [pyfar.dsp.fft](https://pyfar.readthedocs.io/en/stable/modules/pyfar.dsp.fft.html) for a complete documentation.

## References
[J. Ahrens, C. Andersson, P. Höstmad, and W. Kropp, “Tutorial on Scaling of the Discrete Fourier Transform and the Implied Physical Units of the Spectra of Time-Discrete Signals,” AES Convention e-Brief 600, May 2020.](http://www.aes.org/e-lib/browse.cfm?elib=20838)